# RAG를 이용해 다른 데이팅 유저들 유사 사례를 이용해 상담해주기 - 검색 인덱스 만들기

- 대화 데이터 로드하기
- 검색 인덱스 만들고 저장하기
- 검색 인덱스 로드하고 사용하기


In [ ]:
!pip install --upgrade --quiet langchain langchain-openai faiss-cpu tiktoken

In [1]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

import pandas as pd

## 대화 데이터 로드하기

In [2]:
df = pd.read_json("./conv_sent_train.jsonl", lines=True, orient="records")

In [3]:
conv_text_list = []

for i, row in df.iterrows():
    conv_text_list.append(row['messages'][1]['content'])

In [4]:
conv_text_list

['김민준: 안녕하세요, 처음 뵙겠습니다. 김민준이라고 해요.\n이서연: 네, 안녕하세요. 이서연입니다. \n김민준: 요즘 어떻게 지내세요? 취미가 뭐에요?\n이서연: 그냥 그래요. 별로 특별한 취미는 없고요, 여기 와서 할 이야기가 그게 전부인가요?\n김민준: 아니, 그게... 좀 더 서로를 알아가는 과정이라 생각해서요.\n이서연: 솔직히 말해서 여기 오는 것 자체가 별로였어요. 죄송하지만 저는 이만 가볼게요.',
 '김철수: 오늘 날씨가 되게 흐리네요. 우울한 기분이 들어요.\n이하나: 네, 저도 비 오는 날은 싫어요. 기분까지 우울해지는 것 같아요.\n김철수: 그런가요? 저는 뭐든지 긍정적으로 생각하려고 노력하는 편인데, 오늘따라 그게 잘 안되네요.\n이하나: 솔직히 말하면, 저는 이 소개팅 자체가 별로인 것 같아요. 이상하게 잘 안 맞는 것 같아요.',
 '지후: 안녕하세요? 처음 뵙겠습니다. 저는 지후라고 해요.\n하은: 반가워요, 지후 씨. 저는 하은이에요. 오늘 만나게 되어 기쁘네요.\n지후: 하은 씨도 좋은 하루 보내셨나요? 오늘 날씨도 좋고, 기분 좋은 만남이 될 것 같아요.\n하은: 네, 정말 그렇네요. 지후 씨 덕분에 오늘 하루가 더 기분 좋아진 것 같아요.',
 '김민수: 안녕하세요, 처음 뵙겠습니다. 김민수라고 합니다.\n이지은: 네, 안녕하세요. 이지은입니다.\n김민수: 오늘 날씨 참 좋네요. 취미가 뭐에요?\n이지은: 저는 솔직히 소개팅 자체에 별로 관심 없어서요. 취미 생각해본 적도 없고요.',
 '지훈: 오늘 날씨 진짜 안 좋네요.\n은아: 네, 정말 우울하게 만드는 날씨에요.\n지훈: 이렇게 비 오는 날에는 집에만 있고 싶어요.\n은아: 저도요. 근데 왜 이런 날에 소개팅을 한 걸까요?',
 '지훈: 안녕하세요, 처음 뵙겠습니다. 지훈이라고 합니다. 만나서 반가워요.\n민지: 안녕하세요, 민지입니다. 지훈 씨도 만나서 반가워요!\n지훈: 오늘 날씨가 참 좋네요. 이런 날은 야외에서 커피 한 잔하는 것도 좋죠.\n민지: 정말요. 

## 검색 인덱스 만들고 저장하기

In [5]:
vectorstore = FAISS.from_texts(
    conv_text_list, embedding=OpenAIEmbeddings()
)


In [6]:
vectorstore.save_local("./conv_index")

## 검색 인덱스 로드하고 사용하기

In [9]:
vectorstore = FAISS.load_local(folder_path="./conv_index", embeddings=OpenAIEmbeddings(), allow_dangerous_deserialization=True)

In [10]:
retriever = vectorstore.as_retriever()

In [11]:
retriever.invoke("취미")

[Document(id='a4bcb79f-50a7-4eb1-9a3c-862498a6f94e', metadata={}, page_content='지호: 안녕하세요, 처음 뵙겠습니다. 제 이름은 지호예요. 오늘 만나게 되서 정말 기쁩니다!\n민정: 안녕하세요, 저는 민정이라고 해요. 지호 씨도 만나게 되서 반가워요. 오늘 날씨도 좋네요!\n지호: 정말요, 날씨가 우리 만남을 축복해 주는 것 같아요. 민정 씨 취미가 무엇인가요?\n민정: 저는 요리하는 것을 매우 좋아해요. 특히 제철 재료로 건강한 요리를 만드는 것에 흥미가 많아요. 지호 씨는요?'),
 Document(id='044014fe-cd37-453a-9b48-6546cca66109', metadata={}, page_content='지훈: 안녕하세요. 처음 뵙겠습니다. 지훈이라고 합니다.\n민아: 네, 안녕하세요. 민아입니다. 솔직히 말씀드리면, 저 이런 소개팅 자리가 좀 어색하네요.\n지훈: 아, 그렇군요. 저도 사실 잘 모르겠어요. 이런 자리가.\n민아: 일단, 취미가 무엇인가요? 그거라도 이야기해볼까요?\n지훈: 음, 저는 주로 집에서 시간을 보내는 편이라서 특별한 취미는 없어요. 민아 씨는요?\n민아: 저도 특별히 뭐... 크게 다를 건 없네요. 아무래도 이건 좀 잘못된 매칭 같아요, 솔직히 말해서.'),
 Document(id='5796b253-fb04-4659-ae23-ddf1bec17682', metadata={}, page_content='지훈: 안녕하세요, 처음 뵙겠습니다. 지훈이라고 합니다.\n혜지: 반가워요, 저는 혜지에요. 오늘 만나게 되어서 정말 기쁩니다.\n지훈: 혜지 씨 취미가 뭐예요?\n혜지: 저는 요리하는 것을 정말 좋아해요. 지훈 씨는요?'),
 Document(id='a4099f09-3a99-4559-82d3-807ccd72bde3', metadata={}, page_content='김민준: 안녕하세요, 처음 뵙겠습니다. 김민준이라고 해요.\n이서연: 네, 안녕하세요. 이

In [12]:
retriever.invoke("어색")

[Document(id='e614f105-5998-4b93-9a4c-66003a41c21d', metadata={}, page_content='지후: 안녕하세요? 처음 뵙겠습니다. 저는 지후라고 해요.\n하은: 반가워요, 지후 씨. 저는 하은이에요. 오늘 만나게 되어 기쁘네요.\n지후: 하은 씨도 좋은 하루 보내셨나요? 오늘 날씨도 좋고, 기분 좋은 만남이 될 것 같아요.\n하은: 네, 정말 그렇네요. 지후 씨 덕분에 오늘 하루가 더 기분 좋아진 것 같아요.'),
 Document(id='281f5cee-9794-4adc-b62e-50db837eb4eb', metadata={}, page_content='김철수: 오늘 날씨가 되게 흐리네요. 우울한 기분이 들어요.\n이하나: 네, 저도 비 오는 날은 싫어요. 기분까지 우울해지는 것 같아요.\n김철수: 그런가요? 저는 뭐든지 긍정적으로 생각하려고 노력하는 편인데, 오늘따라 그게 잘 안되네요.\n이하나: 솔직히 말하면, 저는 이 소개팅 자체가 별로인 것 같아요. 이상하게 잘 안 맞는 것 같아요.'),
 Document(id='cd063ad8-c6f4-42eb-9a63-26eb76353f4b', metadata={}, page_content='김민수: 안녕하세요, 처음 뵙겠습니다. 김민수라고 합니다.\n이지은: 네, 안녕하세요. 이지은입니다.\n김민수: 오늘 날씨 참 좋네요. 취미가 뭐에요?\n이지은: 저는 솔직히 소개팅 자체에 별로 관심 없어서요. 취미 생각해본 적도 없고요.'),
 Document(id='d74907f1-f420-4e91-b2c2-b8c904d12dbe', metadata={}, page_content='지훈: 안녕하세요, 지훈이라고 합니다. 오늘 처음 뵙게 되어서 반갑습니다.\n소영: 소영입니다. 저도 만나서 반가워요. 지훈 씨는 어떤 일을 하시나요?\n지훈: 저는 IT 회사에서 개발자로 일하고 있어요. 소영 씨는요?\n소영: 저는 디자인 회사에서 근무하고 있습니다. 서로 이렇게 만나게 되어 정

In [13]:
retriever.invoke("별로")

[Document(id='a4099f09-3a99-4559-82d3-807ccd72bde3', metadata={}, page_content='김민준: 안녕하세요, 처음 뵙겠습니다. 김민준이라고 해요.\n이서연: 네, 안녕하세요. 이서연입니다. \n김민준: 요즘 어떻게 지내세요? 취미가 뭐에요?\n이서연: 그냥 그래요. 별로 특별한 취미는 없고요, 여기 와서 할 이야기가 그게 전부인가요?\n김민준: 아니, 그게... 좀 더 서로를 알아가는 과정이라 생각해서요.\n이서연: 솔직히 말해서 여기 오는 것 자체가 별로였어요. 죄송하지만 저는 이만 가볼게요.'),
 Document(id='281f5cee-9794-4adc-b62e-50db837eb4eb', metadata={}, page_content='김철수: 오늘 날씨가 되게 흐리네요. 우울한 기분이 들어요.\n이하나: 네, 저도 비 오는 날은 싫어요. 기분까지 우울해지는 것 같아요.\n김철수: 그런가요? 저는 뭐든지 긍정적으로 생각하려고 노력하는 편인데, 오늘따라 그게 잘 안되네요.\n이하나: 솔직히 말하면, 저는 이 소개팅 자체가 별로인 것 같아요. 이상하게 잘 안 맞는 것 같아요.'),
 Document(id='044014fe-cd37-453a-9b48-6546cca66109', metadata={}, page_content='지훈: 안녕하세요. 처음 뵙겠습니다. 지훈이라고 합니다.\n민아: 네, 안녕하세요. 민아입니다. 솔직히 말씀드리면, 저 이런 소개팅 자리가 좀 어색하네요.\n지훈: 아, 그렇군요. 저도 사실 잘 모르겠어요. 이런 자리가.\n민아: 일단, 취미가 무엇인가요? 그거라도 이야기해볼까요?\n지훈: 음, 저는 주로 집에서 시간을 보내는 편이라서 특별한 취미는 없어요. 민아 씨는요?\n민아: 저도 특별히 뭐... 크게 다를 건 없네요. 아무래도 이건 좀 잘못된 매칭 같아요, 솔직히 말해서.'),
 Document(id='4716ccf6-3ee5-418a-880c-8fc2ae1c2c42', meta